IMPORT AND SETTINGS

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
import matplotlib.pyplot as plt
import pandas as pd
import cv2 
import itertools
import numpy as np
from sklearn.model_selection import train_test_split
import random

try:
    tf.config.experimental.enable_op_determinism()
    print("✅ Op Determinism Abilitato!")
except AttributeError:
    print("⚠️ Attenzione: La tua versione di TF è troppo vecchia per enable_op_determinism.")

def reset_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
SEEDS = [555, 123,42,7,999]

I0000 00:00:1765551689.450296 3442771 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1765551689.480458 3442771 cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1765551690.056336 3442771 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2.21.0-dev20251210
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✅ Op Determinism Abilitato!


W0000 00:00:1765551690.752522 3442771 gpu_device.cc:2456] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0a. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.


DATASET LOADING AND MODELLING

In [2]:
def load_dataset():
    folder='dataset/images'
    data=[]
    for filename in sorted(os.listdir(folder)):
        img_path=os.path.join(folder,filename)
        img=cv2.imread(img_path) #opencv save in bgr
        data.append({
            'image':img,
            'filename':filename
        })

    print(len(data),'images loaded')
    print('file name is: ',data[0]['filename'], 'shape of the image is:  ', data[0]['image'].shape )

    label=pd.read_csv('dataset/raw/bbx_annotations.csv')
    print(label.shape, label.iloc[0]['filename'])
    #images order is random, and for 1 image you can have more class

    print('we have', len(label['class'].unique()), 'different classes')

    #replace biggger img with half sized ones
    #cv2.imwrite('resize_image/last_img_pre_downsampling.jpg',data[-100]['image'])
    for i,item in enumerate(data):
        if "upper" in item["filename"].lower():
            data[i]['image']=cv2.resize(
                data[i]['image'],
                (data[i]['image'].shape[1]//2,data[i]['image'].shape[0]//2)
                ,interpolation=cv2.INTER_AREA
            )
    #cv2.imwrite('resize_image/last_img_post_downsampling.jpg',data[-100]['image'])
       
    return data, label

FROM THE PURE DATASET TO THE TRAIN AND TEST DATA AND LABEL

In [3]:
def dataset_modelling(dataset,annotation):   
    dataset_df = pd.DataFrame(dataset) 
    label_map={'goalpost':0,
               'ball':1,
               'robot':2,
               'goalspot':3,
               'centerspot':4}
    def get_vector(classes_found):
        vec=np.zeros(5,dtype=int)

        for c in classes_found:
            if c in label_map:
                vec[label_map[c]]=1
        return list(vec)
    
    grouped = annotation.groupby('filename')['class'].apply(list).reset_index()
    grouped['label']=grouped['class'].apply(get_vector)
    final_annotation=grouped[['filename','label']]

    final_dataset= pd.merge(dataset_df, final_annotation[['filename', 'label']], on='filename', how='inner')
    final_dataset.to_csv('csv/temp/final_dataset.csv')
    final_dataset=final_dataset.drop(columns=['filename'])
    df_train, df_test = train_test_split(final_dataset, test_size=0.2, random_state=42)
    x_train = np.array(df_train['image'].tolist()).astype('float32') /255.0
    y_train = np.array(df_train['label'].tolist()).astype('float32')
    
    x_test = np.array(df_test['image'].tolist()).astype('float32') / 255.0
    y_test = np.array(df_test['label'].tolist()).astype('float32')
    return x_train, y_train,x_test,y_test
    

DOUBLING THE DATA 

In [4]:
def augment_train_set(x_train,y_train,aug_type):
    rng = np.random.RandomState(42)    
    if aug_type=='flip':
        x_flipped = np.flip(x_train, axis=2)
        y_flipped = y_train
        x_train_aug = np.concatenate([x_train, x_flipped], axis=0)
        y_train_aug = np.concatenate([y_train, y_flipped], axis=0)
    elif aug_type=='noise':
        noise = rng.normal(loc=0.0, scale=0.05, size=x_train.shape)
        x_noisy = x_train + noise
        x_noisy = np.clip(x_noisy, 0., 1.)
        x_train_aug = np.concatenate([x_train, x_noisy], axis=0)
        y_noise = y_train
        y_train_aug = np.concatenate([y_train, y_noise], axis=0)
    elif aug_type=='both':
        x_flipped = np.flip(x_train, axis=2)
        y_flipped = y_train
        noise = rng.normal(loc=0.0, scale=0.05, size=x_train.shape)
        x_noisy = x_train + noise
        x_noisy = np.clip(x_noisy, 0., 1.)
        y_noise = y_train
        x_train_aug = np.concatenate([x_train,x_flipped, x_noisy], axis=0)
        y_train_aug = np.concatenate([y_train,y_flipped, y_noise], axis=0)

    #avoid to have all noisy data in validation--> shuffle
    indices = np.arange(x_train_aug.shape[0])
    rng.shuffle(indices)
    x_train_aug = x_train_aug[indices]
    y_train_aug = y_train_aug[indices]

    return x_train_aug, y_train_aug

MODEL BUILIDNG, 
kernel dimesnsion, pooling dimension, fc layers dimension, number of conv layer and learning rate have different combination.
instead, i fixed:
pooling stride=2 
pooling type: avg pooling
number of kernel per layer: 16, 32, 64...
last pooling: glob avg pool

In [5]:
def build_model():
    model=models.Sequential()
    model.add(tf.keras.Input(shape=(240,320,3)))
    model.add(layers.RandomFlip("horizontal"))
    model.add(layers.GaussianNoise(0.05))

    for i in range(4):
        kernel_number=16*(2**i)
        model.add(layers.Conv2D(kernel_number,(7,7),activation='relu',padding='same'))
        model.add(layers.AveragePooling2D((3,3),strides=2,padding='same'))
    
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dense(5,activation='sigmoid'))

    #model.summary()
    return model


MAIN

In [6]:
dataset, annotation=load_dataset()
x_train,y_train, x_test, y_test=dataset_modelling(dataset, annotation)


for seed in SEEDS:
    bestf1=0
    all_results = []

    print(f"\n ================== INIZIO CICLO CON SEED: {seed} ================== ")

    
    tf.keras.backend.clear_session()
    reset_seeds(seed)
    model=build_model()

    opt=tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                loss='binary_crossentropy',
                metrics=[tf.keras.metrics.Precision(name='precision'),
                        tf.keras.metrics.Recall(name='recall'),
                        ])
    early_stop=EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
    )
    reset_seeds(seed)
    #class_weights_dict=compute_class_weight(y_train)
    history=model.fit(
        x_train,y_train,
        epochs=120,
        batch_size=16, 
        validation_split=0.2, 
        callbacks=[early_stop]
        )

    y_pred=model.predict(x_test)
    predictions_binary = (y_pred > 0.5).astype(int)
    target_names = ['goalpost','ball','robot','goalspot','centerspot']
    report_dict = classification_report(y_test, predictions_binary, target_names=target_names, output_dict=True)
    f1_macro = report_dict['macro avg']['f1-score']
    

    df_report = pd.DataFrame(report_dict).transpose()
    df_report = df_report.round(2)
    df_report['support'] = df_report['support'].astype(int)
    csv_path = f'csv/report/report_7dyn_{seed}.csv'
    df_report.to_csv(csv_path)




2452 images loaded
file name is:  lower_100056_jpg.rf.ec9852c66b4eee4a185317210a378f16.jpg shape of the image is:   (240, 320, 3)
(8125, 8) upper_604302_jpg.rf.6215ee30a829ec658154eb4d067dfdf5.jpg
we have 5 different classes

 ================== INIZIO CICLO CON SEED: 555 ================== 


W0000 00:00:1765551692.709396 3442771 gpu_device.cc:2456] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 12.0a. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
I0000 00:00:1765551692.779440 3442771 gpu_device.cc:2040] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9241 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 5070, pci bus id: 0000:01:00.0, compute capability: 12.0a


Epoch 1/120


I0000 00:00:1765551695.073061 3443095 cuda_dnn.cc:461] Loaded cuDNN version 91600


88/88 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - loss: 0.5252 - precision: 0.7844 - recall: 0.4786 - val_loss: 0.5287 - val_precision: 0.7756 - val_recall: 0.4715
Epoch 2/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - loss: 0.5130 - precision: 0.8075 - recall: 0.4732 - val_loss: 0.5151 - val_precision: 0.7756 - val_recall: 0.4715
Epoch 3/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.4977 - precision: 0.8107 - recall: 0.4777 - val_loss: 0.4909 - val_precision: 0.7756 - val_recall: 0.4715
Epoch 4/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.4851 - precision: 0.8201 - recall: 0.4819 - val_loss: 0.4957 - val_precision: 0.7756 - val_recall: 0.4715
Epoch 5/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.4711 - precision: 0.8298 - recall: 0.4911 - val_loss: 0.4844 - val_precision: 0.7544 - val_recall: 0.5147
Epoch 6/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.4654 - precision: 0.8329 - recall: 0.4977 - val_loss: 0.4909 - val_precision: 0.7592 - val_recall: 0.5009
Epoc

...

/home/simone/Desktop/ml_homework2/ml_hw2/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 1/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - loss: 0.5268 - precision: 0.8018 - recall: 0.4698 - val_loss: 0.5156 - val_precision: 0.7756 - val_recall: 0.4715
Epoch 2/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.5115 - precision: 0.8075 - recall: 0.4732 - val_loss: 0.5110 - val_precision: 0.7756 - val_recall: 0.4715
Epoch 3/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - loss: 0.4830 - precision: 0.8345 - recall: 0.4806 - val_loss: 0.4837 - val_precision: 0.7915 - val_recall: 0.4853
Epoch 4/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - loss: 0.4718 - precision: 0.8351 - recall: 0.4952 - val_loss: 0.5065 - val_precision: 0.7837 - val_recall: 0.4819
Epoch 5/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - loss: 0.4644 - precision: 0.8436 - recall: 0.4960 - val_loss: 0.4759 - val_precision: 0.8033 - val_recall: 0.5009
Epoch 6/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.4592 - precision: 0.8495 - recall: 0.4981 - val_loss: 0.4729 - val_precision: 0.8083 - val_recall:

E0000 00:00:1765552309.870287 3442771 node_def_util.cc:682] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


/home/simone/Desktop/ml_homework2/ml_hw2/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
def visualize_feature_maps(model, input_image):
    layer_outputs = [layer.output for layer in model.layers if 'conv' in layer.name]
    layer_names = [layer.name for layer in model.layers if 'conv' in layer.name]
    activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
    img_batch = np.expand_dims(input_image, axis=0)
    activations = activation_model.predict(img_batch)
    images_per_row = 16
    for layer_name, layer_activation in zip(layer_names, activations):
        # layer_activation ha shape (1, H, W, Num_Filtri)
        n_features = layer_activation.shape[-1] # Numero di filtri (es. 32, 64...)
        size_h = layer_activation.shape[1]      # Altezza feature map
        size_w = layer_activation.shape[2]
        # Limitiamo a 64 feature per non intasare il grafico se il layer è profondo
        n_features = min(n_features, 32) 
        n_cols = n_features // images_per_row
        
        if n_cols == 0: n_cols = 1 # Sicurezza
        
        # Griglia vuota per l'immagine finale
        display_grid = np.zeros((size_h * n_cols, images_per_row * size_w))
        
        for col in range(n_cols):
            for row in range(images_per_row):
                channel_image = layer_activation[0, :, :, col * images_per_row + row]
                
                # Post-process per renderla visibile (normalizzazione)
                channel_image -= channel_image.mean()
                channel_image /= (channel_image.std() + 1e-5) # Evita div by zero
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                
                # Inseriamo nella griglia
                display_grid[col * size_h : (col + 1) * size_h,
                             row * size_w : (row + 1) * size_w] = channel_image

        # Plotting
        scale = 1. / size_h
        plt.figure(figsize=(scale * display_grid.shape[1],
                            scale * display_grid.shape[0]))
        plt.title(f"Layer: {layer_name} ({n_features} feature maps)")
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')
        plt.savefig('images/feature_map/data_aug_fnb.jpg', bbox_inches='tight', dpi=150)
        plt.show()
index_ball = np.where(y_test[:, 1] == 1)[0][0] 
img_to_test = x_test[index_ball]
# 2. Visualizza cosa vede la rete
visualize_feature_maps(model, img_to_test)

